# Кузнецов Михаил Пи19-4


# Оптимальное решение в условиях риска



### Импортирование нужных библиотек

In [1]:
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, LpMaximize
import pandas as pd
import random

#### Метод для ввода пользователем матрицы весовых коэффициентов и названия стратегий с клавиатуры и вектора вероятности

In [62]:
def enterCoef(n, m):
    game_matrix = [ [0]*m for i in range(n) ]
    
    names_of_strategies_a = [0] * n
    names_of_strategies_nature = [0] * m
    # ввод стратегий игрока А
    for i in range(n):
        name = input(f"Введите название  {i+1} стратегии игрока А: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_a[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
     # ввод имен стратегий состояний природы
    for i in range(m):
        name = input(f"Введите название  {i+1} состояния природы: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_nature[i] = name
        else:
            print("Плохой ввод названия состояния")
            raise ValueError
    print("Заполните матрицу весовых коэффициентов")        
    for i in range(n):
        for j in range(m):
            num = int(input(f"Введите элемент строки {i+1} столбца {j+1}: "))
            if num > 0:
                game_matrix[i][j] = num
            else:
                raise ValueError
   
    print("Заполните вектор вероятности")
    P_vector = []
    for i in range(m):
        p = float(input("Введите вероятность события: "))
        if p > 1 or p < 0:
            raise ValueError
        else:
            P_vector.append(p)
         
    return (np.array(game_matrix), names_of_strategies_a, names_of_strategies_nature, np.array(P_vector))

#### Метод для ввода пользователем названия стратегий с клавиатуры и случайная генерация матрицы весовых коэффициентов

In [5]:
def randomCoef(n,m):
    game_matrix = [ [0]*m for i in range(n) ]
    
    names_of_strategies_a = [0] * n
    names_of_strategies_nature = [0] * m
    # ввод стратегий игрока А
    for i in range(n):
        name = input(f"Введите название  {i+1} стратегии игрока А: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_a[i] = name
        else:
            print("Плохой ввод названия стратегии")
            raise ValueError
     # ввод имен стратегий состояний природы
    for i in range(m):
        name = input(f"Введите название  {i+1} состояния природы: ")
        if (name.isdigit() == False) and (len(name) != 0):
            names_of_strategies_nature[i] = name
        else:
            print("Плохой ввод названия состояния")
            raise ValueError
            
    game_matrix = np.random.randint(1, 500, size = (n,m))
    P_vector = np.array([random.random() for i in range(m)])
    return (np.array(game_matrix), names_of_strategies_a, names_of_strategies_nature, np.array(P_vector))

#### Метод для считывания из csv

In [15]:
def readFromScv(path1 = "data/dataA.csv", path2 = "data/P_vector.csv"):
    main_matrix = pd.read_csv(path1)
    names_of_strategies_a = list(main_matrix['Names_a'])
    main_matrix.drop('Names_a', inplace=True, axis=1)
    game_matrix = None
    names_of_strategies_nature = list(main_matrix.columns)
    if sum(main_matrix.isnull().sum()):
        print("Содержатся пропуски!")
        raise ValueError
    else:
        if sum(main_matrix.dtypes == object) > 0:
            print("Содержится строка в файле!")
            raise ValueError
        elif (main_matrix <= 0).any().any() == True:
            print("Содержится отрицательное или нулевое значение!")
            raise ValueError
        else:
            game_matrix = main_matrix.to_numpy()
    
    main_matrix = pd.read_csv(path2)
    P_vector = list(map(float, (main_matrix.columns)))
                
    return (game_matrix, names_of_strategies_a, names_of_strategies_nature, P_vector)

In [56]:
class RiskGame:
    
    def __init__(self, game_matrix, names_strategies_A, names_strategies_nature ,vector_P):
        self.game_matrix = game_matrix
        self.names_strategies_A = names_strategies_A
        self.names_names_strategies_nature = names_strategies_nature
        self.vector_P = vector_P
        
        self.optimal_Laplas = None
        self.cost_optimal_Laplas = None
        
        self.optimal_Bayes = None
        self.cost_optimal_Bayes = None
        
        self.optimal_Hermeyer = None
        self.cost_optimal_Hermeyer = None
        
        self.optimal_mixed_Hermeyer = None
        self.cost_optimal_mixed_Hermeyer = None
        
    
    def simplex_method(self, matrix, F_type, C_type_f):
        model = LpProblem("Game", F_type)
        variables = []

        for i in range(matrix.shape[0]):
            v = LpVariable(f'v{i+1}', lowBound=0)
            variables.append(v)
        
        variables_np = np.array(variables)

        del variables
        
        function = (variables_np).sum()
        model += function
        
        all_constraints = C_type_f((matrix * variables_np).sum(axis=1), 1)
        for c in all_constraints:
            model += c        
        model.solve()
        optimal = np.array(list(map(lambda p: p.value(), model.variables())))
        
        return optimal / optimal.sum(),  1 / optimal.sum()
    

    def Laplas(self):
        lapplas_criterion = (self.game_matrix * (1 / self.vector_P.shape[0])).sum(axis=1)
        lapplas_criterion_opt_v = lapplas_criterion.max()
        lapplas_criterion_opt = np.argwhere(lapplas_criterion == lapplas_criterion_opt_v).reshape(-1)
        lapplas_criterion_opt_strategies = '; '.join(map(lambda x: f'№{x+1} - {self.names_strategies_A[x]}', lapplas_criterion_opt))
        
        self.optimal_Laplas = lapplas_criterion_opt_strategies
        self.cost_optimal_Laplas = lapplas_criterion_opt_v
        
        
    def Bayes(self):
        bayes_criterion = (self.game_matrix * self.vector_P).sum(axis=1)
        bayes_criterion_opt = bayes_criterion.argmax()
        
        self.optimal_Bayes = str(bayes_criterion_opt + 1) + ":" + str(self.names_strategies_A[bayes_criterion_opt])
        self.cost_optimal_Bayes = bayes_criterion[bayes_criterion_opt]
        
        
    def Hermeyer(self):
        hermeyer_criterion = (self.game_matrix * self.vector_P).min(axis=1)
        hermeyer_criterion_opt = hermeyer_criterion.argmax()
        min_j = (self.game_matrix * self.vector_P).argmin(axis=1)[hermeyer_criterion_opt]
        hermeyer_criterion_opt_v = hermeyer_criterion[hermeyer_criterion_opt]

        mixed_opt, mixed_opt_v = self.simplex_method((self.game_matrix * self.vector_P).T, LpMinimize, np.vectorize(lambda x, y: x >= y))
        
        
        
        self.optimal_Hermeyer = str(hermeyer_criterion_opt + 1) + ":" + self.names_strategies_A[hermeyer_criterion_opt]
        self.optimal_mixed_Hermeyer = mixed_opt
        self.cost_optimal_mixed_Hermeyer = str(mixed_opt_v) + "/" + str(((self.game_matrix * self.vector_P).T * mixed_opt).sum())
        self.cost_optimal_Hermeyer = str(hermeyer_criterion_opt_v) + "/" + str(self.game_matrix[hermeyer_criterion_opt, min_j])

    def __str__(self):
        return "Результаты решения\n" + "".join([
        "Optimal of А by Laplas: {}\n".format(self.optimal_Laplas),
        "Cost optimal by Laplas: {}\n".format(self.cost_optimal_Laplas),
        "Optimal of А by Bayes: {}\n".format(self.optimal_Bayes),
        "Cost optimal by Bayes: {}\n".format(self.cost_optimal_Bayes),
        "Optimal of А by Hermeyer: {}\n".format(self.optimal_Hermeyer),
        "Cost optimal by Hermeyer: {}\n".format(self.cost_optimal_Hermeyer),
        "Optimal mixed by Hermeyer: {}\n".format(self.optimal_mixed_Hermeyer),
        "Cost optimal mixed by Hermeyer: {}\n".format(self.cost_optimal_mixed_Hermeyer)])

In [63]:
games = {1:"Ввод с клавиатуры", 2:"Случайная генерация", 3:"Чтение из csv"}

print(f"Выберите один из 3 вариантов получения данных \n Доступные варианты получения данных: {games}")
print()

game_matrix = None
game_names = None

try: 
    choice = int(input("Введите число для получения данных (от 1 до 3): "))
    
    if (choice > 3) or (choice <= 0):
        print("Введите число от 1 до 3!")
    
    else:
        print(f"Выбранный вариант получения данных: {games[choice]}")
        
        if choice == 1:
            try:
                n = int(input("Введите кол-во стратегий для игрока А: "))
                m = int(input("Введите кол-во состояний природы: "))
                
                if (n > 0) and (m > 0):
                    game_matrix_A, names_of_strategies_a, names_of_strategies_nature, P_vector  = enterCoef(n,m)
                else:
                    print("Неправильное кол-во строк/столбцов!")
            except ValueError:
                print("Неправильный ввод!")
            except KeyboardInterrupt:
                 print("Прервано пользователем")
        elif choice == 2:
            try:
                n = int(input("Введите кол-во стратегий для игрока А: "))
                m = int(input("Введите кол-во состояний природы: "))
                
                if (n > 0) and (m > 0):
                    game_matrix_A, names_of_strategies_a, names_of_strategies_nature, P_vector  = randomCoef(n,m)
                else:
                    print("Неправильное кол-во строк/столбцов!")
            except ValueError:
                print("Неправильный ввод!")
            except KeyboardInterrupt:
                print("Прервано пользователем")
        else:
            try:
                game_matrix_A, names_of_strategies_a, names_of_strategies_nature, P_vector = readFromScv()
            except FileNotFoundError:
                print("Файл не найден!")
            except ValueError:
                print("Измените значения ячеек в файле!")
            except KeyboardInterrupt:
                print("Прервано пользователем")
            
except ValueError:
    print("Неправильный ввод!")
except KeyboardInterrupt:
    print("Прервано пользователем")

Выберите один из 3 вариантов получения данных 
 Доступные варианты получения данных: {1: 'Ввод с клавиатуры', 2: 'Случайная генерация', 3: 'Чтение из csv'}

Введите число для получения данных (от 1 до 3): 1
Выбранный вариант получения данных: Ввод с клавиатуры
Введите кол-во стратегий для игрока А: 4
Введите кол-во состояний природы: 4
Введите название  1 стратегии игрока А: Продукты
Введите название  2 стратегии игрока А: Косметика
Введите название  3 стратегии игрока А: Одежда
Введите название  4 стратегии игрока А: Бытовая химия
Введите название  1 состояния природы: Рецессия
Введите название  2 состояния природы: Стагнация
Введите название  3 состояния природы: Оживление
Введите название  4 состояния природы: Подъем
Заполните матрицу весовых коэффициентов
Введите элемент строки 1 столбца 1: 50
Введите элемент строки 1 столбца 2: 30
Введите элемент строки 1 столбца 3: 35
Введите элемент строки 1 столбца 4: 45
Введите элемент строки 2 столбца 1: 40
Введите элемент строки 2 столбца 2:

In [64]:
my_game = RiskGame(game_matrix_A, names_of_strategies_a, names_of_strategies_nature, P_vector)
my_game.Laplas()
my_game.Bayes()
my_game.Hermeyer()
print(my_game)

Результаты решения
Optimal of А by Laplas: №2 - Косметика; №3 - Одежда
Cost optimal by Laplas: 43.75
Optimal of А by Bayes: 2:Косметика
Cost optimal by Bayes: 47.5
Optimal of А by Hermeyer: 3:Одежда
Cost optimal by Hermeyer: 5.25/35
Optimal mixed by Hermeyer: [0.07692307 0.         0.92307693 0.        ]
Cost optimal mixed by Hermeyer: 5.42307680898373/42.76923078487425

